### Import Required Libraries and Set Up Environment Variables

In [1]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [2]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [3]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
# URL, Query begins (?), Parameter name (), Equal sign (=), 
# field name ([@field:fieldname1]), query string operator (&), etc.
nyt_query_url = f"{url}&api-key={nyt_api_key}&fq={filter_query}&sort={sort}&fl={field_list}&begin_date={begin_date}&end_date={end_date}"
# print(nyt_query_url)

In [4]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
for page in range(2):
        # create query with a page number
    # API results show 10 articles at a time
    nyt_query_url = f"{nyt_query_url}&page={page}"
    nyt_query_url
    # Make a "GET" request and retrieve the JSON
    # is the 2nd line necessary or should I combine these? is this even correct?
    response = requests.get(nyt_query_url)
    reviews = response.json()
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(1)
    
    # Try and save the reviews to the reviews_list
    try:
        # reviews_list.extend(reviews["response"]["docs"])

        # loop through the reviews["response"]["docs"] and append each review to the list
        for review in reviews["response"]["docs"]:
            reviews_list.append(review)
            # print(review)
        # Print the page that was just retrieved
        print(f"Checked page {page}")
    except KeyError:
        # Print the page number that had no results then break from the loop
        print(f"No results from page {page}")
        break

Checked page 0
Checked page 1


In [5]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
first_five_results = reviews_list[:5]
json_results = json.dumps(first_five_results, indent=4)
print(json_results)

[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [6]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_list_df = pd.json_normalize(reviews_list)
reviews_list_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
5,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None
6,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,"[{'name': 'subject', 'value': 'Documentary Fil...",2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
7,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
8,https://www.nytimes.com/2023/02/09/movies/your...,This humdrum Netflix romantic comedy features ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-02-10T00:00:05+0000,569,‘Your Place or Mine’ Review: Try Neither,None,None,They Have a Humdrum Kind of Love,None,None,None,By Amy Nicholson,"[{'firstname': 'Amy', 'middlename': None, 'las...",None
9,https://www.nytimes.com/2023/02/02/movies/love...,"To combat the overdose crisis, a group that in...",The New York Times,"[{'name': 'subject', 'value': 'Documentary Fil...",2023-02-02T12:00:11+0000,306,‘Love in the Time of Fentanyl’ Review: Heartbr...,None,None,Love in the Time Of Fentanyl,None,None,None,By Concepción de León,"[{'fir

In [7]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# \u2018 = `
# \u2019 = `
# End string should include " Review" to avoid cutting title early
reviews_list_df["title"] = reviews_list_df["headline.main"].apply(lambda st: st[st.find("\u2018")+1: st.find("\u2019 Review")])
reviews_list_df
# head_line=reviews_list_df["headline.main"]
# head_line

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children
5,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None,One True Loves
6,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,"[{'name': 'subject', 'value': 'Documentary Fil...",2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None,The Lost Weekend: A Love Story
7,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,A Thousand and One
8,https://www.nytimes.com/2023/02/09/movies/your...,This humdrum Netflix romantic comedy features ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-02-10T00:00:05+0000,569,‘Your Place or Mine’ Review: Try Neither,None,None,They Have a Humdrum Kind of Love,None,None,None,By Amy Nicholson,"[{'firstname': 'Amy', 'middlename': None, 'las...",None,Your Place or Mine
9,https://www.nytimes.com/2023/02/02/movies/love...,"To combat the overdose crisis, a group that in...",The

In [8]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
reviews_list_df["keywords"] = reviews_list_df["keywords"].apply(extract_keywords)
reviews_list_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children
5,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies;persons: Bracey, Luke (1989- )...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None,One True Loves
6,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,subject: Documentary Films and Programs;creati...,2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None,The Lost Weekend: A Love Story
7,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,subject: Movies;creative_works: A Thousand and...,2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,A Thousand and One
8,https://www.nytimes.com/2023/02/09/movies/your...,This humdrum Netflix romantic comedy features ...,The New York Times,subject: Movies;creative_works: Your Place or ...,2023-02-10T00:00:05+0000,569,‘Your Place or Mine’ Review: Try Neither,None,None,They Have a Humdrum Kind of Love,None,None,None,By Amy Nicholson,"[{'firstname': 'Amy', 'middlename': None, 'las...",None,Your Place or Mine
9,https://www.nytimes.com/2023/02/02/movies/love...,"To combat the overdose crisis, a group that in...",The New York Ti

In [9]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
# title=reviews_list_df["title"]
# title
title_list=list(reviews_list_df["title"])
title_list

['The Attachment Diaries',
 'What’s Love Got to Do With It?’ Probably a Lo',
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'The Attachment Diaries',
 'What’s Love Got to Do With It?’ Probably a Lo',
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl']

### Access The Movie Database API

In [24]:
# Prepare The Movie Database query
url2 = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

# Query url
tmdb_query_start = f"{url2}{tmdb_key_string}"
# tmdb_query_url2 = f"{url}&api-key={tmdb_api_key}&tl={title_list}"
# print(tmdb_query_url2)


In [26]:
# Create an empty list to store the results
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests

counter = 0

# Loop through the titles
for title in title_list[:]:
    # Check if we need to sleep before making a request               
    # Add 1 to the request counter
    counter += 1
    if counter % 5 == 0:
        time.sleep(1)        
    
    # Perform a "GET" request for The Movie Database
    tmdb_query_url = f"{tmdb_query_start} + &tl={title}"
    movie_response = requests.get(tmdb_query_url)
    movie_data = response.json()
    print(movie_data)
    
    
    
    
        
    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    # try:

        # Get movie id
        # movie_id = requests.get()

        # Make a request for a the full movie details


        # Execute "GET" request with url

        
        # Extract the genre names into a list


        # Extract the spoken_languages' English name into a list


        # Extract the production_countries' name into a list


        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list

        
        # Print out the title that was found



{'status': 'OK', 'copyright': 'Copyright (c) 2024 The New York Times Company. All Rights Reserved.', 'response': {'docs': [{'web_url': 'https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html', 'snippet': 'A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.', 'source': 'The New York Times', 'headline': {'main': '‘The Attachment Diaries’ Review: Love, Sick', 'kicker': None, 'content_kicker': None, 'print_headline': 'The Attachment Diaries', 'name': None, 'seo': None, 'sub': None}, 'keywords': [{'name': 'subject', 'value': 'Movies', 'rank': 1, 'major': 'N'}, {'name': 'creative_works', 'value': 'The Attachment Diaries (Movie)', 'rank': 2, 'major': 'N'}, {'name': 'persons', 'value': 'Diment, Valentin Javier', 'rank': 3, 'major': 'N'}], 'pub_date': '2023-05-25T11:00:03+0000', 'byline': {'original': 'By Jeannette Catsoulis', 'person': [{'firstname': 'Jeannette', 'middlename': None, 'lastname': 'Catsoulis', 'qua

In [12]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data


In [13]:
# Convert the results to a DataFrame


### Merge and Clean the Data for Export

In [14]:
# Merge the New York Times reviews and TMDB DataFrames on title


In [15]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing


# Create a list of characters to remove


# Loop through the list of columns to fix

    # Convert the column to type 'str'


    # Loop through characters to remove


# Display the fixed DataFrame


In [16]:
# Drop "byline.person" column


In [17]:
# Delete duplicate rows and reset index


In [18]:
# Export data to CSV without the index
